In [91]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import classify
import mcc

In [92]:
import htwt
import iris

In [93]:
mvn = stats.multivariate_normal.rvs
y = 1.0 * np.random.choice(2, size = 200)
X = np.array([mvn([1, 1] if heads else [4, 4]) for heads in y])

In [94]:
X, y = htwt.get()

In [95]:
X, y = iris.get()
X = X[:, [0, 1]]

In [98]:
import nb
import lda
import qda
import rda
import nsc
import irls
import logistic
reload(logistic)
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestCentroid

In [99]:
clf = BernoulliNB()
clf = nb.bernoulli()
clf = GaussianNB()
clf = nb.gaussian()
clf = rda.Classifier()
clf = NearestCentroid()
clf = nsc.Classifier()
clf = LDA()
clf = lda.Classifier()
clf = qda.Classifier()
clf = QDA()
clf = irls.Classifier()
clf = logistic.Classifier()
clf = LogisticRegression()

In [100]:
clf.fit(X, y)
print '%0.3f' % np.mean(np.mean(clf.predict(X) != y))

0.233


In [108]:
reload(classify)
from numpy.linalg import svd
U, s, V = svd(X)
print U.dot(np.diag(s)).dot(V) == X
classify.show_points(U[:, 0 : 2], y)
# classify.show_regions(clf, X)
# classify.show_contour(clf, X)
plt.show()

ValueError: shapes (150,150) and (2,2) not aligned: 150 (dim 1) != 2 (dim 0)